In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam

from keras.optimizers.legacy import Adam as LegacyAdam

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pickle as pkl
from src.utils.data_transform import *
import pandas as pd
import os 
import pickle 
import json

In [ ]:
def load_data(test_subject, subject_to_indices):
    training_data, testing_data, validation_data = [], [], []
    training_labels, testing_labels, validation_labels = [], [], []

    # Load training data (all subjects except the test and validation subjects)
    for subject in subject_to_indices.keys():
        if subject != test_subject :
            # and subject != validation_subject:
            subject_data, subject_labels = load_subject_data(f"../data/ProcessedSubjects/MajorityLabel(95%)/subject_{subject}/data.pkl")
            training_data.append(subject_data)
            training_labels.append(subject_labels)

    # Load testing data (only the test subject)
    test_data, test_labels = load_subject_data(f"../data/ProcessedSubjects/subject_{test_subject}/data.pkl")
    testing_data.append(test_data)
    testing_labels.append(test_labels)

    # # Load validation data (only the validation subject)
    # val_data, val_labels = load_subject_data(f"../data/ProcessedSubjects/subject_{validation_subject}/data.pkl")
    # validation_data.append(val_data)
    # validation_labels.append(val_labels)

    # Combine all training, testing, and validation data and labels
    training_data = np.concatenate(training_data, axis=0)
    training_labels = np.concatenate(training_labels, axis=0)
    testing_data = np.concatenate(testing_data, axis=0)
    testing_labels = np.concatenate(testing_labels, axis=0)
    # validation_data = np.concatenate(validation_data, axis=0)
    # validation_labels = np.concatenate(validation_labels, axis=0)

    return training_data, training_labels, testing_data, testing_labels


In [ ]:
def load_subject_data(path):
    data = pd.read_pickle(path)
    signal_data = np.array([item[0] for item in data])
    label_data = np.array([item[1] for item in data])
    return signal_data, label_data

In [ ]:
def build_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))  # Assuming 5 classes for the output layer
    # optimizer = Adam(learning_rate=1e-3)
    optimizer = LegacyAdam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
with open("../data/processed.nosync/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)

subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

In [ ]:
results = []
accuracy = []
loss = []

In [ ]:
for test_subject in subject_to_indices.keys():
    # Load the data
    print(f"Training without {test_subject}")
    model = build_model(input_shape=(20,6))
    train_data, train_labels, test_data, test_labels = load_data(test_subject, subject_to_indices)
    history = model.fit(train_data, train_labels, epochs=32, batch_size=64)
    results.append(model.evaluate(test_data, test_labels))
    accuracy.append(history.history['accuracy'])
    loss.append(history.history['loss'])
    model.save(f"../models/full_loso/model_{test_subject}.keras")

In [ ]:
history.history['accuracy']

In [ ]:
# epochs = range(len(acc))  # Number of epochs

In [ ]:
np.mean(accuracy[0][:])
avg_accuracy = [np.mean(sublist) for sublist in accuracy]
avg_accuracy

In [ ]:
np.mean(avg_accuracy)

In [ ]:
np.mean(loss)

In [ ]:
# # Plotting training and validation accuracy
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(epochs, acc, label='Training Accuracy')
# # plt.plot(epochs, val_acc, label='Validation Accuracy')
# plt.title('Training Accuracy')
# plt.legend()
# 
# # Plotting training and validation loss
# plt.subplot(1, 2, 2)
# plt.plot(epochs, loss, label='Training Loss')
# # plt.plot(epochs, val_loss, label='Validation Loss')
# plt.title('Training Loss')
# plt.legend()
# 
# plt.show()

In [ ]:
# def build_lstm():
#     lstm_model = Sequential()
#     lstm_model.add(TimeDistributed(cnn_model, input_shape=(35, input_shape[0], input_shape[1])))  # Assuming input_shape is (20, 6)
#     lstm_model.add(LSTM(64, activation='tanh', recurrent_activation='hard_sigmoid', return_sequences=True))
#     lstm_model.add(LSTM(64, activation='tanh', recurrent_activation='hard_sigmoid'))
#     lstm_model.add(Dropout(0.5))
#     lstm_model.add(Dense(1, activation='sigmoid'))  # Binary classification
# 
#     lstm_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
#     return lstm_model
